<a href="https://colab.research.google.com/github/ailouislu/property-forecast-system/blob/main/notebooks/property_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

获取 properties 表中 status 不为 NULL 的数据并进行分析和训练

第一步：安装 Supabase 和 scikit-learn

In [102]:
# 安装所需的库
!pip install supabase
!pip install scikit-learn
!pip install python-dotenv

1.导入库和创建 Supabase 客户端

In [103]:
# 导入所需的库
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from supabase import create_client, Client
from dotenv import load_dotenv

# 连接到 Supabase
dotenv_path = '/content/.env'  # 替换为你的 .env 文件路径

# 加载环境变量
load_dotenv(dotenv_path=dotenv_path)

# 获取环境变量
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

def test_database_connection():
    """测试数据库连接是否正常"""
    try:
        # 创建 Supabase 客户端
        supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

        # 执行一个简单的查询来测试连接
        response = supabase.table('properties').select('id').limit(1).execute()

        # 检查查询结果
        if response.data:
            print("数据库连接成功！")
            print("""
            ___  ___      _       _
            |  \/  |     | |     | |
            | .  . | __ _| |_ ___| |__
            | |\/| |/ _` | __/ __| '_ \
            | |  | | (_| | || (__| | | |
            \_|  |_/\__,_|\__\___|_| |_|
            """)  # 打印 logo
            return True
        else:
            print("数据库连接失败：无法获取数据。")
            return False

    except Exception as e:
        print(f"数据库连接失败：{e}")
        return False

# 测试数据库连接
if test_database_connection():
    print("数据库连接测试通过！")
else:
    print("数据库连接测试失败，请检查你的配置。")

数据库连接成功！

            ___  ___      _       _
            |  \/  |     | |     | |
            | .  . | __ _| |_ ___| |__
            | |\/| |/ _` | __/ __| '_             | |  | | (_| | || (__| | | |
            \_|  |_/\__,_|\__\___|_| |_|
            
数据库连接测试通过！


In [104]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from supabase import create_client, Client
from dotenv import load_dotenv
import os
# utils.py
import numpy as np

def to_json_serializable(records):
    """
    将 list[dict] 中的 numpy 数据类型转换为 Python 原生类型，
    以便 json.dumps() 或 Supabase 客户端序列化不报错。
    """
    new_recs = []
    for rec in records:
        new = {}
        for k, v in rec.items():
            if isinstance(v, np.integer):        # 包括 int64, int32…
                new[k] = int(v)                  # 转为 Python int :contentReference[oaicite:0]{index=0}
            elif isinstance(v, np.floating):     # 包括 float64, float32…
                new[k] = float(v)                # 转为 Python float :contentReference[oaicite:1]{index=1}
            else:
                new[k] = v
        new_recs.append(new)
    return new_recs



第二步：读取数据库

In [105]:
def create_supabase_client() -> Client:
    print("正在创建 Supabase 客户端...")
    if not SUPABASE_URL or not SUPABASE_KEY:
        print("错误: Supabase URL 和 API key 未提供")
        raise ValueError("Supabase URL 和 API key 必须提供")
    client = create_client(SUPABASE_URL, SUPABASE_KEY)  # 初始化客户端 :contentReference[oaicite:0]{index=0}
    print("Supabase 客户端创建成功")
    return client

def fetch_training_data():
    """
    获取训练数据：从 properties_with_is_listed（status=1）和 properties_to_predict（status=0）视图中获取数据，并合并为训练集。
    """
    print("开始获取训练数据...")
    client = create_supabase_client()
    try:
        # 获取 status=1 的房产数据
        res_listed = (
            client
            .from_('properties_with_is_listed')
            .select('*')
            .execute()
        )
        df_listed = pd.DataFrame(res_listed.data) if res_listed.data else pd.DataFrame()

        # 获取 status=0 的房产数据
        res_unlisted = (
            client
            .from_('properties_to_predict')
            .select('*')
            .execute()
        )
        df_unlisted = pd.DataFrame(res_unlisted.data) if res_unlisted.data else pd.DataFrame()

        # 合并两个数据集
        df_combined = pd.concat([df_listed, df_unlisted], ignore_index=True)

        if df_combined.empty:
            print("警告: 没有获取到训练数据。")
            return None

        print(f"获取的训练数据列: {df_combined.columns.tolist()}")
        return df_combined

    except Exception as e:
        print(f"错误: 获取训练数据时发生错误: {e}")
        return None


def fetch_prediction_data():
    """
    获取预测数据：查询 properties，
    并可按 suburb 等额外条件过滤。
    """
    print("开始获取预测数据...")
    client = create_supabase_client()
    try:
        res = (
            client
            .from_('properties_to_predict')
            .select('*')
            # .eq('suburb', 'Khandallah')    # 区域过滤
            .execute()
        )
        print("成功从 Supabase 获取预测数据")
        if res.data:
            df = pd.DataFrame(res.data)
            print(f"获取的预测数据列: {df.columns.tolist()}")
            return df
        else:
            print("警告: 没有获取到需要预测的数据。")
            return None
    except Exception as e:
        print(f"错误: 获取预测数据时发生错误: {e}")
        return None


3. 训练

In [106]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

def extract_property_history_features(history_str):
    """
    从 property_history 字段中提取有用的特征：
    1. 交易次数
    2. 最近一次交易距今的天数
    3. 是否包含建造记录
    """
    if pd.isnull(history_str) or not isinstance(history_str, str):
        return 0, -1, 0

    # 提取交易事件
    events = history_str.split('; ')
    transaction_count = len(events)

    # 检查是否包含“Property Built”事件
    has_built_event = int(any('Property Built' in event for event in events))

    # 提取最近一次交易距今的天数
    date_pattern = r'(\d{4}-\d{2}-\d{2})'
    dates = [re.search(date_pattern, event) for event in events]
    dates = [pd.to_datetime(match.group(1)) for match in dates if match]

    if dates:
        most_recent_date = max(dates)
        days_since_last_transaction = (pd.Timestamp.now() - most_recent_date).days
    else:
        days_since_last_transaction = -1  # 没有找到日期时使用默认值

    return transaction_count, days_since_last_transaction, has_built_event

def preprocess_data(df, for_prediction=False):
    print("开始数据预处理...")
    feature_columns = ['year_built', 'bedrooms', 'bathrooms', 'car_spaces', 'floor_size',
                       'land_area', 'last_sold_price', 'capital_value',
                       'land_value', 'improvement_value', 'suburb',
                       'has_rental_history', 'is_currently_rented', 'property_history']

    X = df[feature_columns].copy()

    # 处理数值型特征
    X['floor_size'] = pd.to_numeric(X['floor_size'].replace({'m²': '', ',': ''}, regex=True), errors='coerce')
    X['land_area'] = pd.to_numeric(X['land_area'].replace({'m²': '', ',': ''}, regex=True), errors='coerce')
    X['has_rental_history'] = X['has_rental_history'].astype(int)
    X['is_currently_rented'] = X['is_currently_rented'].astype(int)

    # 提取 property_history 特征
    X['transaction_count'], X['days_since_last_transaction'], X['has_built_event'] = zip(
        *X['property_history'].apply(extract_property_history_features)
    )
    X = X.drop(columns=['property_history'])

    # 处理类别型特征（如 suburb）
    if 'suburb' in X.columns:
        X['suburb'] = X['suburb'].astype('category').cat.codes

    # 将所有特征转为数值型，并填充缺失值
    for col in X.columns:
        X[col] = pd.to_numeric(X[col], errors='coerce')

    # 处理无穷大值，将其替换为数据集中的最大合理值或默认值
    X.replace([float('inf'), -float('inf')], -1, inplace=True)

    X = X.fillna(X.mean())

    if not for_prediction:
        y = df['status'].astype(int)  # 0/1
        # 改为警告而非抛错
        uniques = y.unique()
        print(">> 标签唯一值：", uniques)
        if len(uniques) < 2:
            print("警告：训练集只有单一标签，无法训练有区分度模型，请检查数据视图或查询。")
        return X, y, None
    else:
        return X


def train_model(X, y):
    print("开始模型训练...")
    # 增加 stratify 以保持正负样本比例
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    # 修复语法，增加 class_weight
    model = RandomForestClassifier(
        n_estimators=100,
        class_weight='balanced',
        random_state=42
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"模型准确率: {accuracy:.2f}")

    joblib.dump((model, X.columns.tolist()), 'property_status_model.joblib')
    print("模型和特征名称已保存为 'property_status_model.joblib'")

    return model, X.columns.tolist()


4. 预测并存储

In [107]:
from datetime import datetime

def predict_and_store(model, feature_names, prediction_df, le):
    """
    预测并将结果存储到数据库。
    """
    print("开始预测并存储结果...")

    # 创建 Supabase 客户端
    supabase_client = create_supabase_client()

    # ✅ 插入前清空 property_status 表
    try:
        print("正在清空 property_status 表中的旧数据...")
        delete_result = supabase_client.table('property_status').delete().neq('id', 0).execute()
        print(f"已清空 property_status 表，共删除 {len(delete_result.data) if delete_result.data else 0} 条记录。")
    except Exception as delete_error:
        print(f"警告: 删除旧数据时发生错误 - {delete_error}")

    # 预处理数据
    X_pred = preprocess_data(prediction_df, for_prediction=True)
    X_pred = X_pred.reindex(columns=feature_names, fill_value=0)  # 确保所有列都匹配

    # 预测结果和置信度
    predictions = model.predict(X_pred)
    confidence_scores = model.predict_proba(X_pred).max(axis=1)

     # —— 核心改动：处理 le=None 情况 ——
    if le is not None:
        # 正常使用 LabelEncoder 反编码:contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}
        predicted_statuses = le.inverse_transform(predictions)
    else:
        # 如果没有 le，则直接使用数值标签，或映射为字符串
        # 这里假设 1 表示 'for Sale'，0 表示 'not for Sale'
        mapping = {1: 'for Sale', 0: 'not for Sale'}
        predicted_statuses = [mapping.get(int(p), str(p)) for p in predictions]

    # 当前时间精确到秒
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # 准备要插入的数据
    records_to_insert = []
    for idx, row in prediction_df.iterrows():
        property_id = row['id']
        predicted_status = predicted_statuses[idx]
        confidence_score = float(confidence_scores[idx])

        records_to_insert.append({
            'property_id': property_id,
            'predicted_status': predicted_status,
            'confidence_score': confidence_score,
            'predicted_at': current_time
        })

    # 转换所有 numpy 类型为原生类型
    records_to_insert = to_json_serializable(records_to_insert)

    # 批量插入数据
    try:
        result = supabase_client.table('property_status').insert(records_to_insert).execute()

        # 检查插入结果并记录日志
        if result.data:
            for inserted_record in result.data:
                inserted_id = inserted_record.get('property_id')
                predicted_at = inserted_record.get('predicted_at', current_time)
                print(f"成功存储预测结果 - ID: {inserted_id}, 预测状态: {inserted_record['predicted_status']}, 置信度: {inserted_record['confidence_score']:.4f}, 预测时间: {predicted_at}")
        else:
            print("警告: 批量插入成功，但未能检索到插入的记录。")
    except Exception as insert_error:
        print(f"错误: 批量存储预测结果时发生错误 - 错误详情: {insert_error}")


5. 主程序

In [108]:
def main():
    print("程序开始运行...")
    try:
        training_df = fetch_training_data()
        if training_df is None or training_df.empty:
            print("错误: 没有获取到有效的训练数据，程序终止。")
            return

        X, y, le = preprocess_data(training_df)

        model, feature_names = train_model(X, y)

        prediction_df = fetch_prediction_data()
        if prediction_df is None or prediction_df.empty:
            print("警告: 没有获取到需要预测的数据，程序终止。")
            return

        predict_and_store(model, feature_names, prediction_df, le)
        print("预测完成，结果已存储到数据库。")

    except Exception as e:
        print(f"错误: 程序运行时发生错误：{e}")
        import traceback
        print("错误详情:")
        print(traceback.format_exc())

    print("程序运行结束。")

if __name__ == "__main__":
    main()


程序开始运行...
开始获取训练数据...
正在创建 Supabase 客户端...
Supabase 客户端创建成功
获取的训练数据列: ['id', 'property_address', 'suburb', 'p_status', 'status', 'property_history', 'last_sold_date', 'year_built', 'bedrooms', 'bathrooms', 'car_spaces', 'floor_size', 'land_area', 'last_sold_price', 'capital_value', 'land_value', 'improvement_value', 'has_rental_history', 'is_currently_rented']
开始数据预处理...
>> 标签唯一值： [1 0]
开始模型训练...
模型准确率: 0.96
模型和特征名称已保存为 'property_status_model.joblib'
开始获取预测数据...
正在创建 Supabase 客户端...
Supabase 客户端创建成功
成功从 Supabase 获取预测数据
获取的预测数据列: ['id', 'property_address', 'suburb', 'p_status', 'status', 'property_history', 'last_sold_date', 'year_built', 'bedrooms', 'bathrooms', 'car_spaces', 'floor_size', 'land_area', 'last_sold_price', 'capital_value', 'land_value', 'improvement_value', 'has_rental_history', 'is_currently_rented']
开始预测并存储结果...
正在创建 Supabase 客户端...
Supabase 客户端创建成功
正在清空 property_status 表中的旧数据...
已清空 property_status 表，共删除 1000 条记录。
开始数据预处理...
成功存储预测结果 - ID: 03a26169f9e8a1deb7e72fae9b6c